### Baseline using a simple CountVectorizer

In [9]:
import os, json
import pandas as pd
import numpy as np
#!pip install --upgrade tensorflow
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# running locally
'''BASE_DIR = '../data_pipeline/'

# get data sets 
df_train = pd.read_csv(os.path.join(BASE_DIR, "df_train.csv"), index_col=0)
df_val = pd.read_csv(os.path.join(BASE_DIR, "df_validation.csv"), index_col=0)

total_data = len(df_train) + len(df_val)
train_ratio = len(df_train) / total_data
val_ratio = len(df_val) / total_data

print(f"Train-Val split is: {train_ratio:.2f}-{val_ratio:.2f}")
# split val 0.5:0.5 in val and test

df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)

df_test.head()'''

'BASE_DIR = \'../data_pipeline/\'\n\n# get data sets \ndf_train = pd.read_csv(os.path.join(BASE_DIR, "df_train.csv"), index_col=0)\ndf_val = pd.read_csv(os.path.join(BASE_DIR, "df_validation.csv"), index_col=0)\n\ntotal_data = len(df_train) + len(df_val)\ntrain_ratio = len(df_train) / total_data\nval_ratio = len(df_val) / total_data\n\nprint(f"Train-Val split is: {train_ratio:.2f}-{val_ratio:.2f}")\n# split val 0.5:0.5 in val and test\n\ndf_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)\n\ndf_test.head()'

In [10]:
# running in kaggle

df_train = pd.read_csv('/kaggle/input/author-data/df_train.csv')
df_val = pd.read_csv('/kaggle/input/author-data/df_validation.csv')

# TO DO: DO NOT SPLIT WHEN WE HAVE THE TEST SET! CHANGE HERE
df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)
df_test.head()

# REPLACE WITH ACTUAL TEST SET!

,Unnamed: 0,paragraph1,paragraph2,label_author,label_dataset,n_authors,fileindex
3532,3532,"In general, be courteous to others. Debate/dis...",For those who have questions regarding any med...,0,1,2,205
2899,2899,A paper airplane powered by a nuclear powered ...,"Do I get to mark ""Turns out USA created Corona...",1,1,2,80
3822,3822,Once they have the enriched uranium it is surp...,Its a combination of materials and secrets. Nu...,1,1,4,266
5846,5846,"If it’s not political, being a leftist is irre...","Yes it is. Of course, if you enacted a policy ...",0,1,4,662
9035,9035,"Irrelevant, if confirmed it would mean there w...","This time, it looks like it wasn't Russia so i...",1,2,4,427


In [11]:
# convert labels to numerical values
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['label_author'])
y_val = label_encoder.transform(df_val['label_author'])
y_test = label_encoder.transform(df_test['label_author'])

# get bow representation of paragraph1
vectorizer_par1 = CountVectorizer(max_features=5000)
X_train_par1 = vectorizer_par1.fit_transform(df_train['paragraph1']).toarray()
X_val_par1 = vectorizer_par1.transform(df_val['paragraph1']).toarray()
X_test_par1 = vectorizer_par1.transform(df_test['paragraph1']).toarray()

# get bow representation of paragraph2
vectorizer_par2 = CountVectorizer(max_features=5000)
X_train_par2 = vectorizer_par2.fit_transform(df_train['paragraph2']).toarray()
X_val_par2 = vectorizer_par2.transform(df_val['paragraph2']).toarray()
X_test_par2 = vectorizer_par2.transform(df_test['paragraph2']).toarray()


done
done
done


In [13]:
# Define the features for training, validation, and test  sets
X_train = np.concatenate((X_train_par1, X_train_par2), axis=1)
X_val = np.concatenate((X_val_par1, X_val_par2), axis=1)
X_test = np.concatenate((X_test_par1, X_test_par2), axis=1)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# evaluate on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# evaluate on test set REPLACE WITH ACTUAL TEST SET
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

#test_predictions = model.predict(X_test) # should be list of style changes
model.save("baseline_model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
  56/1625 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6017 - loss: 0.6411

I0000 00:00:1715946270.560979     316 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1715946270.576967     316 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1625/1625 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6610 - loss: 0.5594

W0000 00:00:1715946276.838447     314 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1625/1625 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6610 - loss: 0.5594 - val_accuracy: 0.6603 - val_loss: 0.5364
Epoch 2/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7582 - loss: 0.4502 - val_accuracy: 0.6549 - val_loss: 0.5712
Epoch 3/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9045 - loss: 0.2302 - val_accuracy: 0.6508 - val_loss: 0.8332
Epoch 4/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9856 - loss: 0.0567 - val_accuracy: 0.6535 - val_loss: 1.4251
Epoch 5/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9968 - loss: 0.0190 - val_accuracy: 0.6428 - val_loss: 1.6908
Epoch 6/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9985 - loss: 0.0097 - val_accuracy: 0.6457 - val_loss: 2.0608
Epoch 7/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9972 - loss: 0.0133 - val_accuracy: 0.6494 - val_loss: 2.1097
Epoch 8/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9983 - loss: 0.0084 - val_accura